# Example usage of SeisSrcMoment to calculate $M_W$ in the frequency domain, using a linearised $M_0$ method

This example is for a volcano-tectonic earthquake at Uturuncu, Bolivia. The moment magnitude, $M_W$, is calculated in the frequency domain, i.e. the long period spectral level is calculated by fitting a Brune model, as detailed in Stork et al (2014). However, here we use a linearised Brune model to calculate a number of the parameters in the Brune model with enhanced numerical stability over the non-linear Brune fit method. This earthquake's moment tensor is analysed in Alvizuri and Tape (2016), with $M_W$ = 2.80 found by full waveform moment tensor inversion.

## 1. Specify parameters to use:

In [1]:
import numpy as np
import os, sys 
%load_ext autoreload
%autoreload 2
from SeisSrcMoment import moment_linear_reg
from NonLinLocPy import read_nonlinloc

In [2]:
# Specify variables:
inventory_fname = "data/instrument_gain_data/IRISDMC-Plutons_dataless.dataless"  # The inventory fname, pointing to the dataless file for the network (for full instrument frequency response removal)
# mseed_filename = "data/mseed_data/20100516063454720000.m" # Note: One can pass the script an obspy stream instead if one wishes.
# NLLoc_event_hyp_filename = "data/NLLoc_data/loc.Tom_RunNLLoc000.20100516.063457.grid0.loc.hyp"
stations_not_to_process = []
window_before_after = [0.1, 0.6] # The time before and after the phase pick to use for calculating the magnitude within
filt_freqs = [0.5, 49.0] # Filter frequencies to apply (important if not removing long period spectral noise)
MT_six_tensors = [] # If this is not specified, assumes average DC component in P (or S) from Stork et al (2014).
density = 2750. #2000. # Density of medium, in kg/m3
Vp = 5000. # P-wave velocity in m/s
# Note that Q not required as the program calculates Q when fitting the source model.
verbosity_level = 0 # Verbosity level (1 for moment only) (2 for major parameters) (3 for plotting of traces)
plot_switch = True
remove_noise_spectrum = False # If True, removes noise using spectrum taken from window before trace. Not thoroughly tested yet, but can get around by applying a high pass filter above anyway.
invert_for_geom_spreading = False
freq_inv_intervals = None #np.arange(0.5, 40, 4.0) #None # Frequency intervals to sample the displacement spectra for the inversion.


One also needs to set the inversion method option for the moment parameters to invert for. The options are as follows:

![title](inversion_options_diagram.png)


In [3]:
# Set inversion method option:
inv_option = "method-1" # Options are: "method-1"; "method-2"; "method-3"; "method-4", as above.

In [4]:
# And simulate for multiple events (TESTING!!!):
mseed_fnames_tmp = np.loadtxt("data/mseed_fnames_list.txt", dtype=str)
nlloc_fnames_tmp = np.loadtxt("data/nlloc_fnames_list.txt", dtype=str)
mseed_filenames = []
NLLoc_event_hyp_filenames = []
for i in range(len(mseed_fnames_tmp)):
    mseed_filenames.append(os.path.join("data/mseed_data", mseed_fnames_tmp[i]))
    NLLoc_event_hyp_filenames.append(os.path.join("data/NLLoc_data", nlloc_fnames_tmp[i]))

mseed_filenames = mseed_filenames[0:10]
NLLoc_event_hyp_filenames = NLLoc_event_hyp_filenames[0:10]


## Run moment calculation:

In [5]:
# Find seismic moment release:
event_obs_dicts, linear_moment_inv_outputs = moment_linear_reg.calc_moment_via_linear_reg(NLLoc_event_hyp_filenames, density, Vp, mseed_filenames=mseed_filenames, inventory_fname=inventory_fname, window_before_after=window_before_after, filt_freqs=filt_freqs, stations_not_to_process=stations_not_to_process, MT_six_tensors=MT_six_tensors, verbosity_level=verbosity_level, invert_for_geom_spreading=invert_for_geom_spreading, freq_inv_intervals=freq_inv_intervals, inv_option=inv_option, plot_switch=plot_switch)
# print("Seismic moment release (Nm):", av_M_0)

/Users/eart0504/opt/anaconda3/lib/python3.7/site-packages/obspy/io/xseed/fields.py:377: UserWarning: Date is required.
  warnings.warn('Date is required.', UserWarning)


Skipping station as spectra of length less than usual.


/Users/eart0504/opt/anaconda3/lib/python3.7/site-packages/obspy/io/xseed/fields.py:377: UserWarning: Date is required.
  warnings.warn('Date is required.', UserWarning)
/Users/eart0504/opt/anaconda3/lib/python3.7/site-packages/obspy/io/xseed/fields.py:377: UserWarning: Date is required.
  warnings.warn('Date is required.', UserWarning)
/Users/eart0504/opt/anaconda3/lib/python3.7/site-packages/obspy/io/xseed/fields.py:377: UserWarning: Date is required.
  warnings.warn('Date is required.', UserWarning)
/Users/eart0504/opt/anaconda3/lib/python3.7/site-packages/obspy/io/xseed/fields.py:377: UserWarning: Date is required.
  warnings.warn('Date is required.', UserWarning)
/Users/eart0504/opt/anaconda3/lib/python3.7/site-packages/obspy/io/xseed/fields.py:377: UserWarning: Date is required.
  warnings.warn('Date is required.', UserWarning)
/Users/eart0504/opt/anaconda3/lib/python3.7/site-packages/obspy/io/xseed/fields.py:377: UserWarning: Date is required.
  warnings.warn('Date is required.',

In [6]:
# Look over events, calculating magnitudes:
for NLLoc_event_hyp_filename in NLLoc_event_hyp_filenames:
    # And find corresponding moment magnitude, M_w (Hanks and Kanamori 1979):
    event_obs_dict = event_obs_dicts[NLLoc_event_hyp_filename]
    M_ws = []
    Qs = []
    t_stars = []
    fcs = []
    for station in list(event_obs_dict.keys()):
        M_0 = event_obs_dict[station]['M_0']
        if M_0 > 0: # filter any events that didn't solve for
            M_ws.append((2./3.)*np.log10(M_0) - 6.0)
            Qs.append(event_obs_dict[station]['Q'])
            t_stars.append(event_obs_dict[station]['t_star'])
            fcs.append(event_obs_dict[station]['f_c'])
    if len(M_ws) > 0:
        M_w = np.average(np.array(M_ws))
        Q = np.average(np.array(Qs))
        t_star = np.average(np.array(t_stars))
        fc = np.average(np.array(fcs))
        print("Moment magnitude, Mw:", M_w, "( Q:", Q, ", t*:", t_star, ", fc:", fc, ")")

Moment magnitude, Mw: 1.64349668286 ( Q: 403.398015003 , t*: 0.0139227573034 , fc: 4.22112068915 )
Moment magnitude, Mw: 1.81722636512 ( Q: 2114.08282202 , t*: 0.00716811813051 , fc: 3.3857840782 )
Moment magnitude, Mw: 1.70306433902 ( Q: 541.411137414 , t*: 0.0219437967143 , fc: 5.24764346533 )
Moment magnitude, Mw: 1.67258651943 ( Q: 1508.80164335 , t*: 0.00676540790153 , fc: 3.96669487377 )
Moment magnitude, Mw: 1.87504580262 ( Q: 1015.58574676 , t*: 0.0122022062394 , fc: 3.8491930345 )
Moment magnitude, Mw: 0.875233632658 ( Q: 609.345745153 , t*: 0.00795001859709 , fc: 14.1243171689 )
Moment magnitude, Mw: 2.14226121416 ( Q: 3143.38722478 , t*: 0.0167009490096 , fc: 5.18238068988 )
Moment magnitude, Mw: 1.53200722853 ( Q: 1925.88055958 , t*: 0.00777180110443 , fc: 5.26205894746 )
Moment magnitude, Mw: 1.81760767072 ( Q: 8385.77676317 , t*: 0.00616655527416 , fc: 3.87921680225 )
Moment magnitude, Mw: 1.85131441449 ( Q: 2486.86218767 , t*: 0.00683673126824 , fc: 3.93093145264 )
